In [1]:
import pickle as pkl
import numpy as np
import pandas as pd
import sklearn
from twitter_nlp_toolkit.tweet_sentiment_classifier import tweet_sentiment_classifier
from twitter_nlp_toolkit.file_fetcher import file_fetcher
from zipfile import ZipFile

Using TensorFlow backend.


In [2]:
chunk = 1 # Fraction of data to train on
model_name = 'model_test_05'
redownload=False


Here we download pre-trained models and a validation dataset. The models have been trained on the Sentiment140 dataset, taken form here: https://www.kaggle.com/kazanova/sentiment140

The validation data is hand-labeled airline customer feedback taken from https://www.figure-eight.com/data-for-everyone/

In [3]:
if redownload: 
    downloader = file_fetcher.downloader(using_notebook=True)
    # Validation data
    downloader.download_file('https://www.dropbox.com/s/440m6x07bjg6c0h/Tweets.zip?dl=1',"Tweets.zip")
    
    # Compressed model
    # Note that this model has only been trained on 5% of the training data
    # Update when better-trained model is available
    downloader.download_file("https://www.dropbox.com/s/i88eqlja56xncyx/model_test_05.zip?dl=1","model_test_05.zip")
    
    # Extract all the contents of zip file in current directory
    with ZipFile(model_name + '.zip', 'r') as zipObj:
        zipObj.extractall()

In [4]:
# Load the validation data

test_data = pd.read_csv('Tweets.zip', header=0, names=['Index', 'Sentiment', 'Sentiment_confidence',
                                                                'Negative_reason', 'Negative_reason_confidence',
                                                                'Airline', 'Airline_sentiment_gold', 'Handle',
                                                                'Negative_reason_gold', 'Retweet_count', 'Text',
                                                                'Tweet_coord', 'Time', 'Location', 'Timezone'])

In [5]:
# Remove the unlabeled test data

test_data['Labels'] = (test_data['Sentiment'] == 'positive') * 2
test_data['Labels'] = test_data['Labels'] + (test_data['Sentiment'] == 'neutral') * 1
test_data['Labels'] = test_data['Labels'] / 2

test_data.set_index('Labels')
test_data = test_data[test_data.Labels != 0.5]

In [6]:
# Executing this cell takes about 30s on a laptop

Classifier = tweet_sentiment_classifier.SentimentAnalyzer()
Classifier.load_models(model_name)

BoW model loaded successfully
LSTM model loaded successfully
Pre-trained embedding model loaded successfully


We santiy check the models: 

In [7]:
Classifier.predict(['I am happy', 'I am sad', 'I am cheerful', 'I am mad'])


array([1., 0., 1., 0.])

In [8]:
# Executing this cell takes several minuites on a laptop

predictions = Classifier.predict(test_data['Text'])


We test the model on an airline customer feedback dataset.

In [9]:
print('Test Accuracy:  {:.3f}'.format(sklearn.metrics.accuracy_score(test_data['Labels'], predictions)))
print('Test MCC:  {:.3f}'.format(sklearn.metrics.matthews_corrcoef(test_data['Labels'], predictions)))
sklearn.metrics.confusion_matrix(test_data['Labels'], predictions)

Test Accuracy:  0.785
Test MCC:  0.538


array([[6974, 2204],
       [ 273, 2090]], dtype=int64)

We have accuracy of just under 80%.

To improve our accuracy, we can refine the model on our airline data. The early stopping procedure (enabled by default to use 10% of the training data for validation) should prevent overfitting. We withold 20% for our own validation. 

In [10]:
trainX, testX, trainY, testY = sklearn.model_selection.train_test_split(test_data['Text'], test_data['Labels'])

In [11]:
Classifier.refine(trainX, trainY)
test_predictions = Classifier.predict(testX)


Train on 7789 samples, validate on 866 samples
Epoch 1/250
7789/7789 [==============================] - 3s 443us/step - loss: 0.4227 - acc: 0.8224 - val_loss: 0.2788 - val_acc: 0.8926
Epoch 2/250
7789/7789 [==============================] - 3s 371us/step - loss: 0.3639 - acc: 0.8533 - val_loss: 0.2387 - val_acc: 0.9111
Epoch 3/250
7789/7789 [==============================] - 3s 368us/step - loss: 0.3280 - acc: 0.8776 - val_loss: 0.2124 - val_acc: 0.9273
Epoch 4/250
7789/7789 [==============================] - 3s 364us/step - loss: 0.3128 - acc: 0.8852 - val_loss: 0.2008 - val_acc: 0.9284
Epoch 5/250
7789/7789 [==============================] - 3s 380us/step - loss: 0.3039 - acc: 0.8907 - val_loss: 0.1998 - val_acc: 0.9249
Epoch 6/250
7789/7789 [==============================] - 3s 370us/step - loss: 0.3093 - acc: 0.8918 - val_loss: 0.2153 - val_acc: 0.9203
Epoch 7/250
7789/7789 [==============================] - 3s 370us/step - loss: 0.3379 - acc: 0.8871 - val_loss: 0.2173 - val_acc: 0

7789/7789 [==============================] - 3s 369us/step - loss: 0.2191 - acc: 0.9167 - val_loss: 0.1575 - val_acc: 0.9434
Epoch 61/250
7789/7789 [==============================] - 3s 385us/step - loss: 0.2188 - acc: 0.9168 - val_loss: 0.1627 - val_acc: 0.9434
Epoch 62/250
7789/7789 [==============================] - 3s 371us/step - loss: 0.2256 - acc: 0.9171 - val_loss: 0.1677 - val_acc: 0.9434
Epoch 63/250
7789/7789 [==============================] - 3s 420us/step - loss: 0.2251 - acc: 0.9177 - val_loss: 0.1720 - val_acc: 0.9423
Epoch 64/250
7789/7789 [==============================] - 3s 391us/step - loss: 0.2265 - acc: 0.9153 - val_loss: 0.1753 - val_acc: 0.9411
Epoch 65/250
7789/7789 [==============================] - 3s 375us/step - loss: 0.2289 - acc: 0.9191 - val_loss: 0.1777 - val_acc: 0.9434
Epoch 66/250
7789/7789 [==============================] - 3s 377us/step - loss: 0.2313 - acc: 0.9162 - val_loss: 0.1792 - val_acc: 0.9457
Epoch 67/250
7789/7789 [=======================

C:\Users\strix\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 7789 samples, validate on 866 samples
Epoch 1/250
7789/7789 [==============================] - 5s 696us/step - loss: 1.0570 - acc: 0.7828 - val_loss: 0.8318 - val_acc: 0.8279
Epoch 2/250
7789/7789 [==============================] - 4s 504us/step - loss: 0.8984 - acc: 0.8079 - val_loss: 0.7253 - val_acc: 0.8395
Epoch 3/250
7789/7789 [==============================] - 4s 501us/step - loss: 0.7773 - acc: 0.8236 - val_loss: 0.6544 - val_acc: 0.8510
Epoch 4/250
7789/7789 [==============================] - 4s 501us/step - loss: 0.7096 - acc: 0.8377 - val_loss: 0.5386 - val_acc: 0.8591
Epoch 5/250
7789/7789 [==============================] - 4s 501us/step - loss: 0.6605 - acc: 0.8491 - val_loss: 0.4996 - val_acc: 0.8707
Epoch 6/250
7789/7789 [==============================] - 4s 497us/step - loss: 0.5961 - acc: 0.8579 - val_loss: 0.4592 - val_acc: 0.8741
Epoch 7/250
7789/7789 [==============================] - 4s 499us/step - loss: 0.5596 - acc: 0.8643 - val_loss: 0.4453 - val_acc: 0

7789/7789 [==============================] - 4s 508us/step - loss: 0.1990 - acc: 0.9398 - val_loss: 0.2777 - val_acc: 0.9134
Epoch 61/250
7789/7789 [==============================] - 4s 508us/step - loss: 0.1940 - acc: 0.9404 - val_loss: 0.2764 - val_acc: 0.9145
Epoch 62/250
7789/7789 [==============================] - 4s 524us/step - loss: 0.1907 - acc: 0.9415 - val_loss: 0.2752 - val_acc: 0.9157
Epoch 63/250
7789/7789 [==============================] - 4s 533us/step - loss: 0.1902 - acc: 0.9429 - val_loss: 0.2741 - val_acc: 0.9157
Epoch 64/250
7789/7789 [==============================] - 4s 546us/step - loss: 0.1852 - acc: 0.9424 - val_loss: 0.2732 - val_acc: 0.9157
Epoch 65/250
7789/7789 [==============================] - 4s 536us/step - loss: 0.1826 - acc: 0.9453 - val_loss: 0.2726 - val_acc: 0.9169
Epoch 66/250
7789/7789 [==============================] - 4s 532us/step - loss: 0.1785 - acc: 0.9449 - val_loss: 0.2734 - val_acc: 0.9180
Epoch 67/250
7789/7789 [=======================

In [12]:
print('Test Accuracy:  {:.3f}'.format(sklearn.metrics.accuracy_score(testY, test_predictions)))
print('Test MCC:  {:.3f}'.format(sklearn.metrics.matthews_corrcoef(testY, test_predictions)))
sklearn.metrics.confusion_matrix(testY, test_predictions)

Test Accuracy:  0.913
Test MCC:  0.727


array([[2184,  119],
       [ 133,  450]], dtype=int64)

Now we have accuracies of over 90%! 

In [13]:
Classifier.evaluate(testX, testY)

BoW: 0.899
LSTM: 0.832
Pre-trained embedding: 0.878


{'ensembled': 0.9126819126819127,
 'BoW': 0.8988218988218988,
 'LSTM': 0.8316008316008316,
 'Glove': 0.8776853776853777}

In [14]:
Classifier.predict(['I dont know'])

array([0.])